In [2]:
# install selenium and other resources for crawling data
# !pip install selenium
# install other resources for doing crawling
# !apt install chromium-chromedriver

import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
import json
import re
import sqlite3
from selenium import webdriver
# Set driver for Chrome
options = webdriver.ChromeOptions()
options.add_argument('-headless') # since we run selenium on Google Colab so we don't want a chrome browser opens, so it will run in the background
options.add_argument('-no-sandbox')


     |████████████████████████████████| 911kB 3.8MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 31 not upgraded.
Need to get 83.2 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 89.0.4389.90-0ubuntu0.18.04.2 [1,127 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 89.0.4389.90-0ubuntu0.18.04.2 [73.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH_TO_DB = '/content/gdrive/MyDrive/'
conn = sqlite3.connect(PATH_TO_DB+'tiki.db')
cur = conn.cursor()

Mounted at /content/gdrive


In [6]:


def get_url(url):
  driver = webdriver.Chrome('chromedriver',options=options)        # Define the chrome drivers with setting options we define above    
  driver.get(url)                                             # Open the browser again to get web page
  html_data = driver.page_source                                   # After driver.get() is done, you can get back HTML string by using .page_source
  driver.close()                                                   # Close the driver after retrieving the web page

  soup = BeautifulSoup(html_data, 'html.parser') 

  return soup

def create_categories_table():
  query = """
      CREATE TABLE IF NOT EXISTS categories (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          name VARCHAR(255),
          url TEXT, 
          parent_id INTEGER, 
          create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
      )
  """
  try:
      cur.execute(query)
      conn.commit()
  except Exception as err:
      print('ERROR BY CREATE TABLE', err)
# create_categories_table()

def select_query(query,conn=conn):
    return pd.read_sql_query(query, conn)

main_categories = [{'Name': 'Điện Thoại - Máy Tính Bảng', 'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},
{'Name': 'Điện Tử - Điện Lạnh', 'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'}, 
{'Name': 'Phụ Kiện - Thiết Bị Số', 'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},
{'Name': 'Laptop - Thiết bị IT', 'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},
{'Name': 'Máy Ảnh - Quay Phim', 'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'}, 
{'Name': 'Điện Gia Dụng', 'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'}, 
{'Name': 'Nhà Cửa Đời Sống', 'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'}, 
{'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'}, 
{'Name': 'Đồ chơi, Mẹ & Bé', 'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'}, 
{'Name': 'Làm Đẹp - Sức Khỏe', 'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},
{'Name': 'Xe Máy, Ô tô, Xe Đạp', 'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},
{'Name': 'Hàng quốc tế', 'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'}, 
{'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}]

class Product:
    def __init__(self, inp_name, inp_sku, inp_price, inp_img, inp_purl, inp_reviews, inp_stars, inp_discount, inp_tikinow=None, inp_freeship = None, inp_badge=None, inp_installment=None, inp_gift=None, inp_main_id = None):
        # self.url = inp_url
        # self.parent_id = inp_parent_id
        self.name = inp_name
        self.sku = inp_sku
        self.price = inp_price
        self.img = inp_img
        self.purl = inp_purl
        self.reviews = inp_reviews
        self.stars = inp_stars
        self.discount = inp_discount
        self.tikinow = inp_tikinow
        self.badge = inp_badge
        self.installment = inp_installment
        self.freeship = inp_freeship
        self.gift = inp_gift
        self.main_id = inp_main_id
    def __repr__(self):
         return f"URL: {self.url}, Parent_ID:{self.parent_id}"
    
    def save_product(self):
        query = """
            INSERT INTO products (Product_name, sku, Price, Image, Product_URL, Reviews, Stars, Discount, Tikinow, Freeship, Badge_under_price, Installment, Gift, main_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """

        val = (self.name, self.sku, self.price, self.img, self.purl, self.reviews, self.stars, self.discount, self.tikinow, 
               self.freeship, self.badge, self.installment, self.gift, self.main_id)
        try:
            cur.execute(query, val)
            # self.cat_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)

class Category:
    def __init__(self, name, url, parent_id=None, cat_id=None):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id

    def __repr__(self):
        return f"ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent: {self.parent_id}"

    def save_into_db(self):
        query = """
            INSERT INTO categories (name, url, parent_id)
            VALUES (?, ?, ?);
        """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            self.cat_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)

#extract product info from each page
def extract_tiki_info(url, parent_id = None):
  data = []
  index = 1
  soup = get_url(url)
  # FIND ALL PRODUCT ITEMS
  products = soup.find_all('a', {'class':'product-item'})
  all_script = soup.find_all('script', {'type':'application/ld+json'})
  print("BATCH SIZE:", len(products))

  if (soup.find('div', {'class':'style__StyledNotFoundProductView-sc-1uz0b49-0'})):
    print("END PAGE")
  elif len(products):
    # EXTRACT INFO TO DICTIONARY
    for i in products: 
        d = {'name':'','price':'','product_url':'','image':'', 'product_sku':'',
            'tiki_now':'','freeship':'','review':'','star_rating':'','under_price':'',
            'discount':'','installment':'','gift':'', 'main_id':''}
        try:
            script = all_script[index]
            dict_content = json.loads(script.text)
            d['name']         = dict_content['name']
            d['price']        = dict_content['offers']['price']
            d['product_url']  = dict_content['url']
            d['image']        = dict_content['image']
            d['product_sku']  = dict_content['sku']
            d['tiki_now']     = bool(i.find('div',{'class':'badge-service'}).find('div',{'class':'item'}))        
            d['freeship']     = bool(i.find('div',{'class':'badge-top'}).text == "Freeship")

            if 'aggregateRating' in dict_content:
                d['review']     = float(dict_content['aggregateRating']['reviewCount'])
            else:
                d['review']     = "N/A"

            if 'aggregateRating' in dict_content:
                d['star_rating']     = float(dict_content['aggregateRating']['ratingValue'])
            else:
                d['star_rating']     = "N/A"
            
            d['under_price']  = bool(i.find('div',{'class':'badge-under-price'}).find('div',{'class':'item'}))

            if i.find('div', {'class':'price-discount__discount'}):
                d['discount']   = int(re.sub('[-%]','', i.find('div',{'class':'price-discount__discount'}).text))
            else:
                d['discount']   = "N/A"
            
            d['installment']  = bool(i.find('div',{'class':'badge-benefits'}).img)
            d['gift']         = bool(i.find('div',{'class':'freegift-list'}))

            d['main_id'] = None
            if parent_id != None:
                d['main_id'] = define_main(parent_id)
            print(parent_id)
    
        except Exception as e:
            print(f'Error when scraping: {e}')
        index += 1
        variable = Product(d['name'], d['product_sku'], d['price'], d['image'], d['product_url'], d['review'], d['star_rating'], d['discount'], d['tiki_now'], d['freeship'], d['under_price'], d['installment'], d['gift'], d['main_id'])
        variable.save_product()


def scrape_tiki(base_url, parent_id = None):
  page_num = 1
  main, opt = base_url.split('?')
  
  while page_num < 3:
    page = f'?page={page_num}&'
    url = main+page+opt
    print("url =", url)
    extract_tiki_info(url, parent_id)
    page_num += 1
    sleep(randint(1,2))

  # items = pd.DataFrame(data = result, columns = result[0].keys())
  # items.to_csv("./result2.csv", index=False)

def can_add_to_cat_set(cat_name,save=False):
      if cat_name not in CATEGORY_SET:
        if save:
          CATEGORY_SET.add(cat_name)
          print(f'Added "{cat_name}" to CATEGORY_SET')
        return True
      return False

def get_main_category(main_categories, save_db=False):
  result=[]
  for i in main_categories:
      _=can_add_to_cat_set(i['Name'],save_db)

      main_cat = Category(i['Name'],i['URL'])
      if save_db:
          main_cat.save_into_db()
      result.append(main_cat)
  return result

def get_sub_categories(parent_category, save_db=False):
  parent_url = parent_category.url
  result = []

  try:
      soup = get_url(parent_url)
      for a in soup.find_all('a', {'class':'item item--category '}):
          name = a.text.strip()
          if can_add_to_cat_set(name,save_db): 
            sub_url = a['href']
            cat = Category(name, sub_url, parent_category.cat_id) # we now have parent_id, which is cat_id of parent category
            if save_db:
                cat.save_into_db()
            result.append(cat)
  except Exception as err:
      print('ERROR IN GETTING SUB CATEGORIES:', err)
  return result

def get_all_categories(categories,save_db):
  categories = categories.copy()
  while len(categories):
      cat_to_crawl = categories[0]
      print(f'Getting {cat_to_crawl} sub-categories...')
      sub_categories = get_sub_categories(cat_to_crawl, save_db=save_db)
      print(f'Finished! {cat_to_crawl.name} has {len(sub_categories)} sub-categories')
      categories+=sub_categories
      del categories[0]
def create_products_table():
  query = """
      CREATE TABLE IF NOT EXISTS products (
          id INTEGER PRIMARY KEY AUTOINCREMENT,  
          Product_name TEXT,
          sku VARCHAR,
          Price INTEGER,                                        
          Image TEXT,
          Product_URL TEXT,
          Reviews INTEGER,
          Stars FLOAT,
          Discount TEXT,
          Tikinow TEXT,
          Freeship TEXT,
          Badge_under_price TEXT,
          Installment TEXT,
          Gift TEXT,
          main_id INTEGER,
          create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
      )
  """

  try:
      cur.execute(query)
      conn.commit()
  except Exception as err:
      print('ERROR BY CREATE TABLE', err)
create_products_table()

def define_main(parent_id):
  if parent_id <= 15:
      return parent_id
  query = f"""
  SELECT parent_id
  FROM categories
  WHERE id = {parent_id}
  """
  p_id = int(pd.read_sql_query(query,conn).iloc[0][0])
  return define_main(p_id)

In [ ]:
create_categories_table()
CATEGORY_SET = set()
main_cat_objs = get_main_category(main_categories,save_db=True)
get_all_categories(main_cat_objs,save_db=True)

Added "Điện Thoại - Máy Tính Bảng" to CATEGORY_SET
Added "Điện Tử - Điện Lạnh" to CATEGORY_SET
Added "Phụ Kiện - Thiết Bị Số" to CATEGORY_SET
Added "Laptop - Thiết bị IT" to CATEGORY_SET
Added "Máy Ảnh - Quay Phim" to CATEGORY_SET
Added "Điện Gia Dụng" to CATEGORY_SET
Added "Nhà Cửa Đời Sống" to CATEGORY_SET
Added "Hàng Tiêu Dùng - Thực Phẩm" to CATEGORY_SET
Added "Đồ chơi, Mẹ & Bé" to CATEGORY_SET
Added "Làm Đẹp - Sức Khỏe" to CATEGORY_SET
Added "Xe Máy, Ô tô, Xe Đạp" to CATEGORY_SET
Added "Hàng quốc tế" to CATEGORY_SET
Added "Voucher - Dịch Vụ - Thẻ Cào" to CATEGORY_SET
Getting ID: 988, Name: Điện Thoại - Máy Tính Bảng, URL: https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner, Parent: None sub-categories...


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
cur.execute('DROP TABLE products;')
conn.commit()

In [7]:
cur.execute (''' WITH lowest_sub as 
(SELECT a.url as url, a.id as parent, b.id as child
FROM categories as a
LEFT JOIN categories as b ON a.id = b.parent_id
WHERE child IS NULL)

SELECT url, parent as id
FROM lowest_sub ''')
text = cur.fetchall()
for i in range(len(text)):
    url = text[i][0]
    parent_id = text[i][1]
    scrape_tiki(url, parent_id)

url = https://tiki.vn/dien-thoai-smartphone/c1795?page=1&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
url = https://tiki.vn/dien-thoai-smartphone/c1795?page=2&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
url = https://tiki.vn/dien-thoai-pho-thong/c1796?page=1&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
url = https://tiki.vn/dien-thoai-pho-thong/c1796?page=2&src=c.1789.hamburger_menu_fly_out_banner
BATCH SIZE: 48
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1

KeyboardInterrupt: ignored